In [1]:
!pip install recbole

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 4.3 MB/s 
     |████████████████████████████████| 596 kB 49.3 MB/s 
     |████████████████████████████████| 27.4 MB 1.5 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
import time
import pandas as pd
import numpy as np
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.general_recommender import BPR
from recbole.trainer import Trainer
from recbole.data.interaction import Interaction

In [3]:
!git clone https://github.com/RUCAIBox/RecDatasets

Cloning into 'RecDatasets'...
remote: Enumerating objects: 1034, done.
remote: Counting objects: 100% (197/197), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 1034 (delta 89), reused 173 (delta 85), pack-reused 837
Receiving objects: 100% (1034/1034), 277.95 KiB | 964.00 KiB/s, done.
Resolving deltas: 100% (436/436), done.


In [4]:
!wget http://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip ml-1m.zip -d raw_data

--2022-05-28 07:33:20--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  4.13MB/s    in 1.4s    

2022-05-28 07:33:22 (4.13 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: raw_data/ml-1m/
  inflating: raw_data/ml-1m/movies.dat  
  inflating: raw_data/ml-1m/ratings.dat  
  inflating: raw_data/ml-1m/README   
  inflating: raw_data/ml-1m/users.dat  


In [5]:
!python RecDatasets/conversion_tools/run.py \
--dataset ml-1m \
--input_path raw_data/ml-1m \
--output_path processed_data/ml-1m \
--convert_inter #--convert_item --convert_user

100% 1000209/1000209 [01:51<00:00, 8941.95it/s]


In [6]:
config = Config(model='BPR', dataset='ml-1m')
config['data_path'] = 'processed_data/ml-1m'
config['metrics'].append('MAP')

In [7]:
print(config)


General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = processed_data/ml-1m
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
neg_sampling = {'uniform': 1}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision', 'MAP']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER_ID_FIELD = user_id
ITEM_ID_FIELD = item_id
RATING_FIELD = rating
TIME_FIELD = timestam

In [8]:
dataset = create_dataset(config)
train_data, valid_data, test_data = \
  data_preparation(config, dataset)

In [9]:
model = \
  BPR(config, train_data.dataset).to(config['device'])

In [10]:
trainer = Trainer(config, model)
best_valid_score, best_valid_result = \
  trainer.fit(train_data, valid_data)

In [11]:
test_result = trainer.evaluate(test_data)

In [12]:
test_result

OrderedDict([('recall@10', 0.1647),
             ('mrr@10', 0.449),
             ('ndcg@10', 0.2593),
             ('hit@10', 0.746),
             ('precision@10', 0.2028),
             ('map@10', 0.1544)])

In [13]:
user_feature = \
  dataset.get_user_feature().to(config['device'])
item_feature = \
  dataset.get_item_feature().to(config['device'])

In [14]:
print(user_feature, item_feature)

The batch_size of interaction: 6041
    user_id, torch.Size([6041]), cuda, torch.int64

 The batch_size of interaction: 3707
    item_id, torch.Size([3707]), cuda, torch.int64




In [15]:
topk = 10
rank_list = [i+1 for i in range(topk)]

In [16]:
# 推薦結果を格納するテーブルを作成する
df_recommend_list = pd.DataFrame(
  columns=['user_id', 'item_id', 'score', 'rank']
)

# 各ユーザに対して、トップ10アイテムを絞り込む
for user_id in user_feature['user_id']:
  item_list = item_feature['item_id']
  user_list = \
    [user_id.to('cpu').detach().numpy()] * \
    len(item_feature)

  df_predict = pd.DataFrame()
  df_predict['user_id'] = user_list
  df_predict['item_id'] = \
    item_list.to('cpu').detach().numpy()
  # IDの補正
  df_predict['user_id'] += 1
  df_predict['item_id'] += 1
  
  # 対象ユーザ×全アイテムのインタラクションデータを作成 
  all_inter = Interaction({
    'user_id': user_id.unsqueeze(0), 
    'item_id': item_list
  })
  # スコアの推論
  prediction = model.full_sort_predict(all_inter)
  df_predict['score'] = \
    prediction.to('cpu').detach().numpy()
  # スコアでの並び替え、トップ10で絞り込み
  df_recommend = df_predict.sort_values(
    'score', ascending=False
  )[:topk]
  df_recommend['rank'] = rank_list

  # 全体テーブルに格納する
  df_recommend_list = \
    df_recommend_list.append(
      df_recommend, 
      ignore_index=True
    )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [17]:
df_recommend_list

,user_id,item_id,score,rank
0,1,2393,0.154529,1
1,1,503,0.154065,2
2,1,938,0.149972,3
3,1,1705,0.140344,4
4,1,2141,0.139466,5
...,...,...,...,...
60405,6041,1299,6.028157,6
60406,6041,1243,5.922610,7
60407,6041,1586,5.891199,8
60408,6041,1117,5.762080,9
